# **Conditional Diffusion With Multi-Headed Attention**

Most of this NB is a copy of `24_diffusion_unet`.

The key differences come from the introduction of **Multi-Headed Attention** and components related to **Conditional Diffusion**. 

In [ ]:
import os

In [ ]:
from miniai.imports import *

from einops import rearrange
from fastprogress import progress_bar

In [ ]:
torch.set_printoptions(precision=4, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray_r'
mpl.rcParams['figure.dpi'] = 70

import logging
logging.disable(logging.WARNING)

set_seed(42)
if fc.defaults.cpus>8: fc.defaults.cpus=8

In [ ]:
xl,yl = 'image','label'
name = "fashion_mnist"
bs = 512
dsd = load_dataset(name)

In [ ]:
# The key difference with the original NB i.e. number 24 is that we are reverting to cosine scheduling due to its
# superior results over Karras et al.
def abar(t): return (t*math.pi/2).cos()**2

def inv_bar(x): return x.sqrt().acos()*2 / math.pi

def noisify(x0):
    device = x0.device
    n = len(x0)
    t = torch.rand(n,).to(x0).clamp(0, 0.999)
    ε = torch.randn(x0.shape, device=device)
    abar_t = abar(t).reshape(-1, 1, 1, 1).to(device)
    xt = abar_t.sqrt()*x0 + (1-abar_t).sqrt() * ε
    return (xt, t.to(device)), ε

def collate_ddpm(b): return noisify(default_collate(b)[xl])

def dl_ddpm(ds): return DataLoader(ds, batch_size=bs, collate_fn=collate_ddpm, num_workers=4)

In [ ]:
@inplace
def transformi(b): b[xl] = [F.pad(TF.to_tensor(o), (2,2,2,2))-0.5 for o in b[xl]]

tds = dsd.with_transform(transformi)
dls = DataLoaders(dl_ddpm(tds['train']), dl_ddpm(tds['test']))

dl = dls.train
(xt,t), eps = b = next(iter(dl))

## **Train Model (with updated code blocks)**

In [ ]:
def timestep_embedding(tsteps, emb_dim, max_period=10000):
    exponent = -math.log(max_period) * torch.linspace(0, 1, emb_dim//2, device=tsteps.device)
    emb = tsteps[:, None].float() * exponent.exp()[None, :]
    emb = torch.cat([emb.sin(), emb.cos()], dim=-1)
    return F.pad(emb, (0, 1, 0, 0)) if emb_dim%2==1 else emb

In [ ]:
# Renaming unet_conv to pre_conv
def pre_conv(ni, nf, ks=3, stride=1, act=nn.SiLU, norm=None, bias=True):
    layers = nn.Sequential()
    if norm: layers.append(norm(ni))
    if act : layers.append(act())
    layers.append(nn.Conv2d(ni, nf, stride=stride, kernel_size=ks, padding=ks//2, bias=bias))
    return layers

In [ ]:
def upsample(nf): return nn.Sequential(nn.Upsample(scale_factor=2.), nn.Conv2d(nf, nf, 3, padding=1))

In [ ]:
# Pre-activation linear version
def lin(ni, nf, act=nn.SiLU, norm=None, bias=True):
    layers = nn.Sequential()
    if norm: layers.append(norm(ni))
    if act : layers.append(act())
    layers.append(nn.Linear(ni, nf, bias=bias))
    return layers

In [ ]:
# Problematic version, skip in favour of the next cell block
class SelfAttention(nn.Module):
    def __init__(self, ni, attn_chans):
        super().__init__()
        self.attn = nn.MultiheadAttention(ni, ni//attn_chans, batch_first=True)
        self.norm = nn.BatchNorm2d(ni)

    def forward(self, x):
        n,c,h,w = x.shape
        x = self.norm(x).view(n, c, -1).transpose(1, 2)
        x = self.attn(x, x, x, need_weights=False)[0]
        return x.transpose(1,2).reshape(n,c,h,w)

In [ ]:
# This is a 1-D self attention since we're not using .view() in forward()
# This attention is not ResNet specific 
class SelfAttention(nn.Module):
    def __init__(self, ni, attn_chans, transpose=True):
        super().__init__()
        self.nheads = ni//attn_chans # number of attention channels
        self.scale = math.sqrt(ni/self.nheads) # for scale, divide by the effective number of heads
        self.norm = nn.LayerNorm(ni)
        self.qkv = nn.Linear(ni, ni*3)
        self.proj = nn.Linear(ni, ni)
        self.t = transpose
    
    def forward(self, x):
        n,c,s = x.shape
        if self.t: x = x.transpose(1, 2)
        x = self.norm(x)
        x = self.qkv(x)
        x = rearrange(x, 'n s (h d) -> (n h) s d', h=self.nheads)
        q,k,v = torch.chunk(x, 3, dim=-1)
        s = (q@k.transpose(1,2))/self.scale
        x = s.softmax(dim=-1)@v
        x = rearrange(x, '(n h) s d -> n s (h d)', h=self.nheads)
        x = self.proj(x)
        if self.t: x = x.transpose(1, 2)
        return x

In [ ]:
# This bit converts 1-D attention to 2-D and back.
class SelfAttention2D(SelfAttention):
    def forward(self, x):
        n, c, h, w = x.shape
        return super().forward(x.view(n, c, -1)).reshape(n, c, h, w)

In [ ]:
class EmbResBlock(nn.Module):
    def __init__(self, n_emb, ni, nf=None, ks=3, act=nn.SiLU, norm=nn.BatchNorm2d, attn_chans=0):
        super().__init__()
        if nf is None: nf = ni
        self.emb_proj = nn.Linear(n_emb, nf*2)
        self.conv1 = pre_conv(ni, nf, ks, act=act, norm=norm)
        self.conv2 = pre_conv(nf, nf, ks, act=act, norm=norm)
        self.idconv = fc.noop if ni==nf else nn.Conv2d(ni, nf, 1)
        self.attn = False
        # If attention is called then we can create a SelfAttention2D layer.
        if attn_chans: self.attn = SelfAttention2D(nf, attn_chans)

    def forward(self, x, t):
        inp = x
        x = self.conv1(x)
        emb = self.emb_proj(F.silu(t))[:, :, None, None]
        scale,shift = torch.chunk(emb, 2, dim=1)
        x = x*(1+scale) + shift
        x = self.conv2(x)
        x = x + self.idconv(inp)
        if self.attn: x = x + self.attn(x) # Add attention
        return x

In [ ]:
def saved(m, blk):
    m_ = m.forward

    @wraps(m.forward)
    def _f(*args, **kwargs):
        res = m_(*args, **kwargs)
        blk.saved.append(res)
        return res

    m.forward = _f
    return m

In [ ]:
class DownBlock(nn.Module): # Adding attn_chans
    def __init__(self, n_emb, ni, nf, add_down=True, num_layers=1, attn_chans=0):
        super().__init__()
        self.resnets = nn.ModuleList([saved(EmbResBlock(n_emb, ni if i==0 else nf, nf, attn_chans=attn_chans), self)
                                      for i in range(num_layers)])
        self.down = saved(nn.Conv2d(nf, nf, 3, stride=2, padding=1), self) if add_down else nn.Identity()

    def forward(self, x, t):
        self.saved = []
        for resnet in self.resnets: x = resnet(x, t)
        x = self.down(x)
        return x

In [ ]:
class UpBlock(nn.Module): # Same case with upblock
    def __init__(self, n_emb, ni, prev_nf, nf, add_up=True, num_layers=2, attn_chans=0):
        super().__init__()
        self.resnets = nn.ModuleList(
            [EmbResBlock(n_emb, (prev_nf if i==0 else nf)+(ni if (i==num_layers-1) else nf), nf, attn_chans=attn_chans)
            for i in range(num_layers)])
        self.up = upsample(nf) if add_up else nn.Identity()

    def forward(self, x, t, ups):
        for resnet in self.resnets: x = resnet(torch.cat([x, ups.pop()], dim=1), t)
        return self.up(x)

In [ ]:
class EmbUNetModel(nn.Module): 
    # In addition to attn_chans, we need to specify the index block where attn needs to be applied first. So we add attn_start.
    # Attention is never applied to the initial (0) layer as it can explode memory usage. Another approach is to specify the grid
    # size where attention needs to begin. 
    def __init__(self, in_channels=3, out_channels=3, nfs=(224, 448, 672, 896), num_layers=1, attn_chans=8, attn_start=1):
        super().__init__()
        self.conv_in = nn.Conv2d(in_channels, nfs=[0], kernel_size=3, padding=1)
        self.n_temb = nf = nfs[0]
        n_emb = nf*4
        self.emb_mlp = nn.Sequential(lin(self.n_temb, n_emb, norm=nn.BatchNorm1d),
                                     lin(n_emb, n_emb))
        self.downs = nn.ModuleList()
        n = len(nfs)
        for i in range(n):
            ni = nf
            nf = nfs[i]
            # No attention if start cond. isn't met.
            self.downs.append(DownBlock(n_emb, ni, nf, add_down=i!=n-1, num_layers=num_layers,
                                        attn_chans=0 if i<attn_start else attn_chans))
        self.mid_block = EmbResBlock(n_emb, nfs[-1]) # Missing attention!
        
        rev_nfs = list(reversed(nfs))
        nf = rev_nfs[0]
        self.ups = nn.ModuleList()
        for i in range(n):
            prev_nf = nf
            nf = rev_nfs[i]
            ni = rev_nfs[min(i+1, len(nfs)-1)]
            self.ups.append(UpBlock(n_emb, ni, prev_nf, nf, add_up=i!=n-1, num_layers=num_layers+1,
                                    attn_chans=0 if i>=n-attn_start else attn_chans))
        self.conv_out = pre_conv(nfs[0], out_channels, act=nn.SiLU, norm=nn.BatchNorm2d, bias=False)

    def forward(self, inp): # Forward remains the same.
        x, t = inp
        temb = timestep_embeddings(t, self.n_temb)
        emb = self.emb_mlp(temb)
        x = self.conv_in(x)
        saved = [x]
        for block in self.downs: x = block(x, emb)
        saved += [p for o in self.downs for p in o.saved]
        x = self.mid_block(x, emb)
        for block in self.ups: x = block(x, emb, saved)
        return self.conv_out(x)

In [ ]:
lr = 1e-2
epochs = 25
opt_func = partial(optim.Adam, eps=1e-5)
tmax = epochs * len(dls.train)
sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax)
cbs = [DeviceCB(), ProgressCB(plot=True), MetricsCB(), BatchSchedCB(sched), MixedPrecision()]
model = EmbUNetModel(in_channels=1, out_channels=1, nfs=(32,64,128,256), num_layers=2)
learn = Learner(model, dls, nn.MSELoss(), lr=lr, cbs=cbs, opt_func=opt_func)

In [ ]:
learn.fit(epochs)

TODO: A note on model performance 

Tanishq had an interesting observation regarding the spatial mixing aspects of multi-headed attention i.e. using attention very early on explodes memory usage due to the model's emphasis on individual pixels. However, we do need attention to be applied _early enough_ to do the required mixing of spatial information to improve performance. The art is in finding the optimal balance, as with all things, between targetted early mixing and memory conservation.

Johno followed up with a great point (there are so many of these going around throughout the course) on a trick used in vision transformers / diffusion transformers, which requires the flattening of an `8x8` patch of an image using a convolutional block to a `1x1xNC` (NC being a large number of channels). This reduces the spatial information by increasig the number of channels, and gets the image down to a manageable size where the application of attention doesn't become as memory intensive.

Another trick is **Patching**, which Perplexity defines as:

> Patching is a crucial technique in the processing of images for neural networks, particularly in architectures like Vision Transformers (ViTs) and Convolutional Neural Networks (CNNs). This method involves segmenting an image into smaller, manageable pieces or "patches," which are then transformed into embeddings suitable for further processing by the network. 

> ### 1. Purpose of Patching
> 
> The primary purposes of patching in neural networks include:
>
>    **Image Segmentation**: Patching divides an image into smaller sections, allowing the model to focus on localized features rather than processing the entire image at once. This segmentation is vital for capturing detailed information that might be lost in larger contexts.
> 
>    **Dimensionality Transformation**: Each patch is transformed into a lower-dimensional representation (embedding). This transformation helps in reducing computational complexity and allows for more efficient processing by the neural network.
> 
>    **Normalization**: After embedding, layer normalization can be applied to stabilize training and improve convergence rates, ensuring that the embeddings maintain consistent statistical properties throughout training.
> 
> ### 2. Patch Embedding Process
> 
>The process of creating patch embeddings typically involves several steps:
>
>    **Patch Extraction**: The input image is divided into non-overlapping patches. For example, an image might be split into `16×16` pixel patches. If the original image size is `224×224`, this results in 196 patches.
>
>    **Flattening and Projection**: Each patch is flattened into a one-dimensional vector and then projected into a higher-dimensional space using a linear transformation (often implemented as a convolution operation). This projection creates a fixed-size embedding for each patch.
> 
>    **Adding Positional Information**: Since transformers do not inherently understand spatial relationships, positional embeddings are added to each patch embedding to retain information about the original spatial arrangement of patches within the image.
>
> ### Applications and Benefits
> 
>    **Improved Feature Learning**: By focusing on smaller patches, models can learn more nuanced features that contribute to better performance in tasks such as image classification and segmentation2.
> 
>    **Scalability**: Patching allows models to handle larger images without requiring disproportionate increases in computational resources. It enables efficient training on high-resolution images by breaking them down into smaller components.
> 
>    **Adaptability to Different Resolutions**: Techniques like Multi-Scale Patch Embedding (MSPE) enhance model adaptability to varying input resolutions by using multiple variable-sized kernels, which can be particularly beneficial in real-world scenarios where image sizes are inconsistent.

## **Sampling (same as before)**

In [ ]:
from miniai.fid import ImageEval

In [ ]:
cmodel = torch.load('models/data_aug2.pkl')
del(cmodel[8])
del(cmodel[7])

@inplace
def transformi(b): b[xl] = [F.pad(TF.to_tensor(o), (2,2,2,2))*2-1 for o in b[xl]]

bs = 2048
tds = dsd.with_transform(transformi)
dls = DataLoaders.from_dd(tds, bs, num_workers=fc.defaults.cpus)

dt = dls.train
xb,yb = next(iter(dt))

ie = ImageEval(cmodel, dls, cbs=[DeviceCB()])

In [ ]:
sz = (2048,1,32,32)

In [ ]:
def ddim_step(x_t, noise, abar_t, abar_t1, bbar_t, bbar_t1, eta, sig, clamp=True):
    sig = ((bbar_t1/bbar_t).sqrt() * (1-abar_t/abar_t1).sqrt()) * eta
    x_0_hat = ((x_t-(1-abar_t).sqrt()*noise) / abar_t.sqrt())
    if clamp: x_0_hat = x_0_hat.clamp(-1,1)
    if bbar_t1 <= sig**2+0.01: sig=0.  # set to zero if very small or NaN
    x_t = abar_t1.sqrt()*x_0_hat + (bbar_t1-sig**2).sqrt()*noise
    x_t += sig * torch.randn(x_t.shape).to(x_t)
    return x_0_hat, x_t

In [ ]:
@torch.no_grad()
def sample(f, model, sz, steps, eta=1., clamp=True):
    model.eval()
    ts = torch.linspace(1 - 1/steps, 0, steps)
    x_t = torch.randn(sz).cuda()
    preds = []
    for i,t in enumerate(progress_bar(ts)):
        t = t[None].cuda()
        abar_t = abar(t)
        noise = model((x_t, t))
        abar_t1 = abar(t-1/steps) if t>=1 / steps else torch.tensor(1)
        x_0_hat, x_t = f(x_t, noise, abar_t, abar_t1, 1-abar_t, 1-abar_t1, eta, 1 - ((i+1)/100), clamp=clamp)
        preds.append(x_0_hat.float().cpu())
    return preds

In [ ]:
preds = sample(ddim_step, model, sz, steps=100, eta=1.)
s = (preds[-1] * 2)
s.min(), s.max(), s.shape

In [ ]:
show_images(s[:25].clamp(-1,1), imsize=1.5)

In [ ]:
ie.fid(s),ie.kid(s),s.shape

In [ ]:
preds = sample(ddim_step, model, sz, steps=100, eta=1.)
ie.fid(preds[-1]*2)

In [ ]:
preds = sample(ddim_step, model, sz, steps=50, eta=1.)
ie.fid(preds[-1]*2)

In [ ]:
preds = sample(ddim_step, model, sz, steps=50, eta=1.)
ie.fid(preds[-1]*2)

## **Moving To A Conditional Model**

We will now have the ability to specify which types / classes of outputs our model should generate.

In [ ]:
def collate_ddpm(b):
    b = default_collate(b)
    (xt, t), eps = noisify(b[xl])
    # The y-label yl will allow us to set the category of outputs
    return (xt, t, b[yl]), eps

In [ ]:
@inplace
def transformi(b): b[xl] = [F.pad(TF.to_tensor(o), (2,2,2,2))-0.5 for o in b[xl]]

tds = dsd.with_transform(transformi)
dls = DataLoaders(dl_ddpm(tds['train']), dl_ddpm(tds['test']))

dl = dls.train
(xt,t,c),eps = b = next(iter(dl))

In [ ]:
class CondUNetModel(nn.Module):
    def __init__(self, n_classes, in_channels=3, out_channels=3, nfs=(224, 448, 672, 896), num_layers=1):
        super().__init__()
        self.conv_in = nn.Conv2d(in_channels, nfs[0], kernel_size=3, padding=1)
        self.n_temb = nf = nfs[0]
        n_emb = nf * 4
        # Convert the labels into a embedding vector, n_emb will ensure it is te same size as the timestep embedding
        self.cond_emb = nn.Embedding(n_class, n_emb)
        self.emb_mlp = nn.Sequential(lin(self.n_temb, n_emb, norm=nn.BatchNorm1d),
                                     lin(n_emb, n_emb))
        self.downs = nn.ModuleList()
        for i in range(len(nfs)):
            ni = nf
            nf = nfs[i]
            self.downs.append(DownBlock(n_emb, ni, nf, add_down=i!=len(nfs)-1, num_layers=num_layers))
        self.mid_block = EmbResBlock(n_emb, nfs[-1])

        rev_nfs = list(reversed(nfs))
        nf = rev_nfs[0]
        self.ups = nn.ModuleList()
        for i in range(len(nfs)):
            prev_nf = nf
            nf = rev_nfs[i]
            ni = rev_nfs[min(i+1, len(nfs)-1)]
            self.ups.append(UpBlock(n_emb, ni, prev_nf, nf, add_up=i!=len(nfs)-1, num_layers=num_layers+1))
        self.conv_out = pre_conv(nfs[0], out_channels, act=nn.SiLU, norm=nn.BatchNorm2d, bias=False)

    def forward(self, inp):
        # Inputs now contain the label (0-9 for fashion mnist), in addition to the activations and timesteps
        x, t, c = inp
        temb = timestep_embedding(t, self.n_temb) # timestep embedding as before
        cemb = self.cond_emb(c) # labels passed to conditioned embedding
        emb = self.emb_mlp(temb) + cemb # time embedding passed to the mlp, then added with cemb resulting in a joing embedding
        x = self.conv_in(x)
        saved = [x]
        for block in self.downs: x = block(x, emb)
        saved += [p for o in self.downs for p in o.saved]
        x = self.mid_block(x, emb)
        for block in self.ups: x = block(x, emb, saved)
        return self.conv_out(x)

In [ ]:
lr = 1e-2
epochs = 25
opt_func = partial(optim.Adam, eps=1e-5)
tmax = epochs * len(dls.train)
sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax)
cbs = [DeviceCB(), ProgressCB(plot=True), MetricsCB(), BatchSchedCB(sched), MixedPrecision()]
model = CondUNetModel(10, in_channels=1, out_channels=1, nfs=(32,64,128,256), num_layers=2)
learn = Learner(model, dls, nn.MSELoss(), lr=lr, cbs=cbs, opt_func=opt_func)

In [ ]:
learn.fit(epochs)

In [ ]:
sz = (256,1,32,32)

In [ ]:
@torch.no_grad()
def cond_sample(c, f, model, sz, steps, eta=1.):
    ts = torch.linspace(1-1/steps,0,steps)
    x_t = torch.randn(sz).cuda()
    # Create a vector containing the conditional target, repeated across the batch
    c = x_t.new_full((sz[0],), c, dtype=torch.int32)
    preds = []
    for i,t in enumerate(progress_bar(ts)):
        t = t[None].cuda()
        abar_t = abar(t)
        # The model learns to denoise by type c
        noise = model((x_t, t, c)) # Pass the c vector to the model
        abar_t1 = abar(t-1/steps) if t>=1/steps else torch.tensor(1)
        x_0_hat,x_t = f(x_t, noise, abar_t, abar_t1, 1-abar_t, 1-abar_t1, eta, 1-((i+1)/100))
        preds.append(x_0_hat.float().cpu())
    return preds

In [ ]:
lbls = dsd['train'].features[yl].names
lbls

In [ ]:
set_seed(42)
cid = 0
preds = sample(cid, ddim_step, model, sz, steps=100, eta=1.)
s = (preds[-1]*2)
show_images(s[:25].clamp(-1,1), imsize=1.5, suptitle=lbls[cid])

In [ ]:
set_seed(42)
cid = 0
preds = sample(cid, ddim_step, model, sz, steps=100, eta=0.)
s = (preds[-1]*2)
show_images(s[:25].clamp(-1,1), imsize=1.5, suptitle=lbls[cid])